# Singlelayer Perceptron

### Descrição:
Nesse notebook iremos utilizar a arquitetura do <b>Singlelayer Perceptron</b> para realizar uma <i>multiclass classification</i> para o famoso problema do Iris-Dataset.

O Singlelayer Perceptron, uma Rede Neural Artificial, possui exatamente os mesmos algoritmos de uma Regressão Logística. Para exemplificarmos seu potencial, desenvolvemos um problema de classificação multiclasse para expressar seu funcionamento e motivar a preparação para o Multilayer Perceptron. No problema de <i>multiclass classification</i>, devemos sempre nos preocupar em enumerar nossas classes e aplicar o <i>one-hot encoding</i>.

<b> Obs.: </b> todas as matrizes/vetores utilizados na fundamentação teórica são consideradas como Vetores-Colunas. A implementação pode diferir um pouco dessa convenção.


## Bibliotecas e Configurações

In [7]:
# -*- coding: utf-8 -*-
%matplotlib qt5

# Libraries
import numpy as np 
import matplotlib.pyplot as plt
import random as rnd

## Modelo Perceptron

O Singlelayer Perceptron difere da Regressão Logística (em sua convenção) por utilizar uma diferente "função logística",  a <b>Função Sinal</b>, que possui a seguinte regra:

$$
    \varphi(x) = \begin{cases} 1 & \text{ if } x \geq 0 \\  0 & \text{ if } x < 0 \end{cases}
$$

No entanto, essa função possui algumas dificuldades de implementação, além de ser vulnerável ao problema de dupla-classificação. Logo, continuaremos utilizando a função logística Sigmoide como função ativadora.

$$
    \varphi(x) = \frac{1}{1+e^{-x}}
$$

In [8]:
# Definição das Funções Sigmoide e Modelo Perceptron
def sigmoid(X):
    ''' Returns the Sigmoid Function applied to a vector (or value) x '''
    return 1 / (1 + np.exp(-1 * X))
    
def h_theta(X, theta):
    ''' Apply the Linear Model for features X and parameters theta '''
    return sigmoid(np.dot(np.transpose(theta), X))

# Teste da Sigmoide
testSigmoid = np.linspace(-30, 30)

plt.figure(1)
plt.title("Sigmoid Function")
plt.plot(testSigmoid, sigmoid(testSigmoid), 'b-')
plt.show()

## One-Hot Encoding

Os Datasets não trarão as classes já configuradas em One-Hot Encoding. Ao invés disso, encontraremos muitos datasets que as classes serão enumeradas (de $1..k$). Esse é o caso do Iris-Dataset.

Criaremos, então, uma função que receba tal enumeração e a transforme em uma matriz One-Hot Encoding.

In [9]:
def oneHotEncoding(y):
    y = y.astype(int)
    
    classes = np.max(y)
    examples = np.size(y,0)
    oneHot = np.zeros([classes, examples])
    
    for j in range(examples):
        oneHot[y[j]-1, j] = 1
        
    return oneHot
        
# Teste
y = np.array([1, 3, 4, 5 ,1, 1, 2, 3, 4, 2, 5])

y = oneHotEncoding(y)
print(y)

[[ 1.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  1.]]


## Função de Predição

No nosso modelo, nossas predições estarão no formato <i>one-hot encoding</i>. No entanto, como estamos usando a Sigmoide, nossas predições terão valores dentro do intervalo [0,1]. Para realizar a classificação corretamente, iremos escrever uma função que irá verifiar, para cada exemplar, qual classe ele terá mais probabilidade de pertencer.

In [10]:
# Definição da Função de OneHotEncoding
def predictOHE(y):
    classes  = np.size(y,0)
    examples = np.size(y,1)
    
    oneHot = np.zeros([classes, examples])
    
    for j in range(examples):
        maxPos = np.argmax(y[:,j], axis=0)
        oneHot[maxPos,j] = 1
    
    return oneHot
    
# Teste da Função de OneHotEncoding
x_ohe = np.array([[-10, 10, 7],
                  [  5, -8,-3],
                  [  2,  0, 1]])

theta_ohe = np.array([[0.1, 0.3, 0.2],
                      [0.7, 0.4, 0.5],
                      [0.1, 0.5, 0.2]])

y_pred_ohe = h_theta(x_ohe, theta_ohe)
print("Before Prediction:\n", y_pred_ohe)
print("")

y_pred_ohe = predictOHE(y_pred_ohe)
print("After Prediction:\n", y_pred_ohe)

Before Prediction:
 [[ 0.93702664  0.0099518   0.21416502]
 [ 0.5         0.450166    0.80218389]
 [ 0.7109495   0.11920292  0.52497919]]

After Prediction:
 [[ 1.  0.  0.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]]


## Função de Acurácia

Em problemas de Classificação, subtrair o valor previsto do valor real não nos dá uma informação muito precisa (sobre, por exemplo, a gravidade do erro). Para avaliar melhor a qualidade de um Treinamento de Classificação existem diversas medidas. Uma bem comum, e simples, consiste na acurácia: a quantidade de exemplos "corretamente classificados". Vamos definir essa porcentagem como sendo:

$$ Acc(\theta) = 100 \times (1 - \frac{1}{m} \sum (h(\theta) - y)^{2}) $$

<b>Obs.:</b> a acurácia só será correta caso tanto $h(\theta)$ quanto $y$ sejam valores binários. Logo, iremos arredondar os resultados de $h(\theta)$ utilizando a função <b>Predict()</b>.

In [11]:
def accuracyFunction(X, y, theta):
    ''' Calculates the percentage of correct classifications '''
    Y_pred = predictOHE(h_theta(X, theta))
    
    return 100 * (1 - (1 / np.size(y)) * np.sum((Y_pred - y) ** 2))

## Extração de Features

Na célula abaixo, definimos a função que será responsável por extrair o conjunto de <i>features</i> de todo o Dataset. Utilizaremos essa função pois, com ela, podemos aumentar a complexidade do nosso modelo ao criar novos atributos que sejam transformações não-lineares dos <i>features</i> do Dataset.

É importante notar que caso o Dataset já possua $n$ <i>features</i>, cada transformação será aplicada às $n$ <i>features</i> e incluida no conjunto, implicando num crescimento de atributos igual a $(Kn)$, onde $K$ representa o grau de complexidade do polinômio dessa transformação.

In [12]:
# Definição da Função de Extração
def featureExtraction(data, n_examples, complexity=1):
    ''' Extracts the features from a dataset and apply polynomial transformations '''
    x = np.ones(n_examples)
    
    for i in range(0, complexity):
        x = np.vstack([x, np.transpose(data[:, 0:-1]) ** (i+1)])
    
    return x

# Teste da Função
dataExtract = np.array([[2, 3, 0],
                        [5, 6, 0],
                        [8, 9, 0]])

x_extract = featureExtraction(dataExtract, 3, 1)
print("Features with complexity 1:\n", x_extract)
print("")

x_extract = featureExtraction(dataExtract, 3, 3)
print("Features with complexity 3:\n", x_extract)

Features with complexity 1:
 [[ 1.  1.  1.]
 [ 2.  5.  8.]
 [ 3.  6.  9.]]

Features with complexity 3:
 [[   1.    1.    1.]
 [   2.    5.    8.]
 [   3.    6.    9.]
 [   4.   25.   64.]
 [   9.   36.   81.]
 [   8.  125.  512.]
 [  27.  216.  729.]]


## Feature Scaling

Na célula abaixo criamos uma função para <b>normalizar</b> as nossas <i>features</i>. O objetivo, como mostrado, é evitar problemas de divergência e permitir um melhor treinamento por meio dos hiperparâmetros.

A fórmula que usaremos para o <i>Feature Scaling</i> será:

$$
    X = \frac{X - \bar{X}}{\sigma(X)}
$$

In [13]:
# Definição da Função de Normalização
def normalizeData(data):
    ''' Apply Feature Scaling to the features set '''
    for i in range(1, np.size(data,0)):
        data[i,:] = (data[i,:]-np.mean(data[i,:])) / np.std(data[i,:])
    
    return data

# Teste da Função
x_norm = np.array([[1., 1., 1., 1., 1.],
                   [2., 9., 4., 7., 5.],
                   [-5.,-4.,-2.,-9.,-8.]])

x_norm = normalizeData(x_norm)
print("Features normalizados:\n",x_norm)

Features normalizados:
 [[ 1.          1.          1.          1.          1.        ]
 [-1.40693001  1.4896906  -0.57932412  0.66208471 -0.16552118]
 [ 0.23284516  0.62092042  1.39707095 -1.31945589 -0.93138063]]


## Linha de Decisão

Na célula abaixo iremos definir uma função simples para calcular e retornar os eixos cartesianos da Linha de Decisão estimada pelo modelo. Essa função apenas utiliza os parâmetros $\theta$ para calcular o coeficiente angular e linear da reta.

In [14]:
def decisionBound(theta, x):
    ''' Calculates and returns a linear Decision Boundary from the model '''
    boundary_X = np.linspace(min(x[1,:]), max(x[1,:]))
    boundary_Y = -1*(theta[1] / theta[2]) * boundary_X - (theta[0] / theta[2]);
    return [boundary_X, boundary_Y]

In [25]:
def feedFoward(X, Theta1, Theta2):
    

    a1 = h_theta(X, Theta1)
    n_H = np.size(Theta1, 1)
    print(a1)
    print(np.size(a1,0), np.size(a1,1))
    
    H = np.vstack([np.ones(n_H), a1])
    a2 = h_theta(H, Theta2)
    
    return [a1, a2]

feedFoward(x,Theta1,Theta2)

[[ 0.5  0.5  0.5 ...,  0.5  0.5  0.5]
 [ 0.5  0.5  0.5 ...,  0.5  0.5  0.5]
 [ 0.5  0.5  0.5 ...,  0.5  0.5  0.5]
 ..., 
 [ 0.5  0.5  0.5 ...,  0.5  0.5  0.5]
 [ 0.5  0.5  0.5 ...,  0.5  0.5  0.5]
 [ 0.5  0.5  0.5 ...,  0.5  0.5  0.5]]
10 150


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [16]:
def backPropagation(a1, a2, y, Theta1, Theta2):
    m = np.size(y)
    
    delta_2 = a2 - y
    delta_1 = np.dot(np.transpose(Theta2), delta_2) * (a2) * (1 - a2)
                     
    grad1 = (1/m) * (delta_1 * a1)
    grad1 = grad1[:,1:]
    
    grad2 = (1/m) * (delta_2 * a2)
    
    return [grad1, grad2]
    

## Programa Principal (Regressão Linear)

No programa principal, iremos programar a Regressão Linear propriamente dita.
Dividimos o código em três partes:

### Part 1: Data Pre-Processing

Nesse trecho, iremos nos preocupar em carregar e organizar o dataset que utilizaremos no treinamento. É nesse momento, também, que iremos declarar e separar as variáveis que definem o Conjunto de Atributos, o Conjunto de Saída e os Parâmetros do Modelo, além dos Hiperparâmetros de Treinamento. Iremos seguir a convenção de considerar todos os exemplares como vetores-colunas. No entanto, o numpy não nos permite facilmente modificar essa informação para o Conjunto de Saída, e o mesmo continuará como vetor-linha (sem muito prejuízo). Iremos criar, também um vetor para armazenar o Histórico de Erros do treinamento (por motivos de visualização).

Iremos utilizar o dataset <i>data1.txt</i> localizado na pasta <i>datasets/</i>. Teremos as seguintes matrizes:

$$
    X = \begin{bmatrix} 1 & 1 & \cdots & 1 \\  X_{1}^{(1)} & X_{1}^{(2)} & \cdots & X_{1}^{(m)}  \end{bmatrix};\   \theta = \begin{bmatrix} \theta_{0} \\ \theta_{1}\end{bmatrix};\  Y = \begin{bmatrix} Y^{(1)} & Y^{(2)} & \cdots & Y^{(m)} \end{bmatrix}
$$

### Part 2: Linear Regression Training

Para cada época até a convergência (ou até atingir o limite máximo definido pelo Hiperparâmetro) iremos realizar o Treinamento da Regressão Linear. Os passos serão os seguintes:

1. Calculamos o vetor de predição "Y_pred", como resultado da predição do Modelo para os parâmetros daquela época;
2. Utilizando "Y_pred", calculamos os erros de acordo com o a matriz real "Y";
3. Concatenamos o Custo Total do erro calculado no Histórico de Erros;
4. Realizamos, para cada parâmetro, o Gradiente Descendente para estimar os novos valores dos parâmetros;
5. Imprimimos os resultados do treino a cada 500 épocas;
6. Verificamos uma possível convergência do treino, e paremos o mesmo caso seja verificado;

### Part 3: Data Plotting and Training Results

Ao fim do treinamento, iremos plotar duas figuras para avaliar o resultado final do nosso algoritmo. A <b>Figura 1</b> irá apenas exibir os atributos do Dataset. A <b>Figura 2</b> irá exibir a função estimada pelo nosso Modelo Linear, além do Histórico de Erros dado as épocas até convergência.

In [21]:
# Main Function
if __name__=='__main__':
    
    ###############################
    # Part 1: Data Pre-Processing #
    ###############################
    # Loads the data
    data = np.loadtxt("../datasets/irisDataset.txt")
    
    n_examples = np.size(data, 0)

    # Defines the hyperparameters and training measurements
    alfa = 1
    gamma = 0
    max_epochs = 500000

    neurons_Hidden = 10
    
    error_hist = np.zeros([max_epochs])
    epsilon = 0.001
    
    # Define the model parameters
    x = normalizeData(featureExtraction(data, n_examples, 1))    
    y = oneHotEncoding(data[:, -1])
    
    n_features = np.size(x, 0)
    
    Theta1 = np.zeros([n_features, neurons_Hidden])
    Theta2 = np.zeros([neurons_Hidden, 1])

    ######################################
    # Part 2: Linear Regression Training #
    ######################################
    for epochs in range(max_epochs):
        # Randomly shuffle the data
        randomIndex = rnd.sample(range(n_examples), n_examples)
        x = x[:, randomIndex]
        y = y[:, randomIndex]
        
        print(feedFoward(x,Theta1,Theta2))
        break
        
        # Perform Gradient Descent
        for k in range(n_classes):
            for j in range(n_features):
                for m in range(n_examples):
                    if (j >= 1):
                        theta[j,k] = theta[j,k] - (alfa/n_examples) * ((error[k,m] * x[j,m]) + (gamma * theta[j,k]))
                    else:
                        theta[j,k] = theta[j,k] - (alfa/n_examples) * (error[k,m] * x[j,m])

        # Prints training status at each 100 epochs
        if(epochs % 50 == 0):
            print("###### Epoch", epochs, "######")
            print("Error:", error_hist[epochs])
            print("Thetas:\n", theta)
            print("")
        
        # Evaluate convergence and stops training if so
        if(abs(error_hist[epochs] - error_hist[epochs-50]) <= epsilon):
            print("Gradient Converged!!!\nStopping at epoch", epochs)
            print("###### Epoch", epochs, "######")
            print("Error:", error_hist[epochs])
            print("Thetas:\n", theta)
            print("")
            break

    ######################################
    # Part 3: Visualizing Classification #
    ######################################
    predictions = predictOHE(h_theta(x, Theta2))
    
    print("Results:")
    print("### Predictions ###")
    for i,j in zip(predictions, range(n_classes)):
        print("\nClass",j)
        print(i)
        
    print("\n### Real ###")
    for i,j in zip(y, range(n_classes)):
        print("\nClass",j)
        print(i)
#__

ValueError: all the input array dimensions except for the concatenation axis must match exactly

## Plotagem dos Dados

In [70]:
#############################################
# Part 4: Data Plotting and Training Result #
#############################################
# First Figure: Dataset plotting
iSetos = np.where(y[0,:] == 1)
iVersi = np.where(y[1,:] == 1)
iVirg = np.where(y[2,:] == 1)

deciBound = np.array([decisionBound(theta[:,0], x), decisionBound(theta[:,1], x), decisionBound(theta[:,2], x)])

plt.figure(2)
plotCount = 1
for i in range(1,4):
    for j in range(i+1,5):
        plt.subplot(2,3,plotCount)
        plt.title("Iris Dataset Classification\n(Green=Iris-setosa; Blue=Iris-versicolor; Red=Iris-virginica)")
        plt.grid()
        plt.plot(x[i,iSetos], x[j,iSetos], 'go',
                 x[i,iVersi], x[j,iVersi], 'bo',
                 x[i,iVirg],  x[j,iVirg],  'ro',
                 deciBound[0,0], deciBound[0,1], 'k-',
                 deciBound[1,0], deciBound[1,1], 'k-')
        plotCount += 1

plt.show()

In [10]:
data = np.loadtxt("../datasets/dataMLP1.txt")

pos = np.where(data[:, 2] == 1)
neg = np.where(data[:, 2] == 0)

plt.figure(4)
plt.plot(data[pos,1], data[pos,0], 'go', data[neg,1], data[neg,0], 'bo')
plt.show()